# LangChain Simple Agent

This notebook demonstrates how to roughly use LangChain to create a simple agent.

## Setup and Installation

First, let's install the required packages and verify the installation:

In [ ]:
# Install required packages
!pip install langchain openai python-dotenv langchain_community

# Verify installations
import importlib

def check_package(package_name):
    try:
        importlib.import_module(package_name)
        return True
    except ImportError:
        return False

packages = {
    'langchain': 'LangChain',
    'openai': 'OpenAI',
    'dotenv': 'python-dotenv',
    'langchain_community': 'LangChain Community'
}

all_installed = True
for package, display_name in packages.items():
    installed = check_package(package)
    print(f"{display_name}: {'✅ Installed' if installed else '❌ Not installed'}")
    all_installed = all_installed and installed

if all_installed:
    print("\n✅ All required packages are installed!")
else:
    print("\n⚠️ Some packages are missing. Run the installation command again.")

## Environment Setup

Load environment variables from the `.env` file. <br>
N.b. it will look through the entire project for a valid `.env` file.

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API keys from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Set environment variables for compatibility with libraries that expect them
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY or ""

# Verify API keys are set
if not OPENAI_API_KEY:
    print("⚠️ Warning: OPENAI_API_KEY is not set in .env file")
else:
    print("✅ API keys are set")

## Build a Conversation chain

The simplest way to use LangChain is to make a conversation chain, which you can see below is not unlike many chats available to discuss with LLMs.

In this code, we will define our agent's model and then try making a conversation with it.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory

# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini")

# Define a callable for session history
def get_session_history_callable(session_id: str):
    return InMemoryChatMessageHistory()

# Create a conversation runnable with message history
conversation = RunnableWithMessageHistory(
    runnable=llm,
    get_session_history=get_session_history_callable
)

# Start a conversation
try:
    response = conversation.invoke(
        {"input": "What is a pink elephant ?"},
        config={"configurable": {"session_id": "test-session"}}
    )
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")


## Define Tools
Now that we have a working language model with memory, let’s take it further by building an Agent.

Agents in LangChain allow the model to reason about actions and interact with tools dynamically (like a calculator, search engine, or custom function).

You must first define what functionalities the agent shall have. In this case, let's make a writing agent, capable of writing different kinds of texts based on the provided prompts. This agent will have access to three different tool to help in its writing:

- A `Write Blog` tool, that will create a blog post about a topic
- A `Summarize Text` tool
- A `Generate Title and Intro` tool

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.tools import Tool

# Define tools
tools = [
    Tool(
        name="Write Blog",
        func=lambda topic: llm.invoke(f"Write a detailed and engaging blog post about: {topic}"),
        description="Writes a blog post about the given topic."
    ),
    Tool(
        name="Summarize Text",
        func=lambda text: llm.invoke(f"Summarize this into bullet points:\n\n{text}"),
        description="Summarizes text into concise bullet points."
    ),
    Tool(
        name="Generate Title and Intro",
        func=lambda topic: llm.invoke(f"Create a blog post title and intro paragraph for: {topic}"),
        description="Generates a catchy title and an engaging introduction paragraph."
    )
]


## Create the Agent

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor, initialize_agent

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True
)


## Define Agent State Schema

For more advanced agents, it's important to define a state schema to track the agent's state during execution.
This helps manage the agent's memory, decisions, and outputs in a structured way.

In [ ]:
from typing import TypedDict, List, Dict, Optional

# Define our state schema
class AgentState(TypedDict):
    # The input query from the user
    query: str
    # Tool choices and results
    tool_name: Optional[str]
    tool_input: Optional[str]
    tool_output: Optional[str]
    # Tracking history
    history: List[Dict]
    # Final output
    final_output: Optional[str]

# Example initialization of the agent state
initial_state: AgentState = {
    "query": "",
    "tool_name": None,
    "tool_input": None,
    "tool_output": None,
    "history": [],
    "final_output": None
}

# Demonstrate how to update state during agent execution
def update_agent_state(state: AgentState, **kwargs) -> AgentState:
    """Helper function to update agent state"""
    new_state = state.copy()
    for key, value in kwargs.items():
        if key in new_state:
            new_state[key] = value
    return new_state

# Example of using the state in an agent workflow
def agent_workflow_example():
    # Initialize state with a query
    state = update_agent_state(initial_state, query="Write about apple pies")
    
    # Simulate tool selection
    state = update_agent_state(state, 
                              tool_name="Write Blog",
                              tool_input="evolution of apple pies")
    
    # Simulate tool execution and result
    tool_result = "Apple pies have evolved over centuries..."
    state = update_agent_state(state, tool_output=tool_result)
    
    # Add interaction to history
    state = update_agent_state(state, 
                              history=state["history"] + [{
                                  "tool": state["tool_name"],
                                  "input": state["tool_input"],
                                  "output": state["tool_output"]
                              }])
    
    # Set final output
    state = update_agent_state(state, final_output=f"Here's information about apple pies: {tool_result}")
    
    return state

# Run example workflow
example_state = agent_workflow_example()
print("Example Agent State:")
for key, value in example_state.items():
    print(f"{key}: {value if key != 'history' else len(value)} items")

## Test the Agent

In [ ]:
# Use the agent and handle potential parsing errors
try:
    blog_post = agent.run(
        input="Write a blog post about the evolution of apple pies.",
        handle_parsing_errors=True
    )
    print(blog_post)
except ValueError as e:
    print(f"An error occurred: {e}")

# Now, using the previous prompt, let's make another query to the agent
try:
    title_intro = agent.run(
        input="Define the title and intro for this blog post: " + blog_post,
        handle_parsing_errors=True
    )
    print(title_intro)
except ValueError as e:
    print(f"An error occurred: {e}")

# Now, let's use the agent to summarize the blog post we just created
try:
    summary = agent.run(
        input="Summarize the blog post into bullet points: " + blog_post,
        handle_parsing_errors=True
    )
    print(response)
except ValueError as e:
    print(f"An error occurred: {e}")

# Now let's assemble the blog post using the title, intro, and summary
print("\nAssembled Blog Post: \n Title and Intro: \n" + title_intro + "\n\n Excerpt: \n" + summary + "\n\n Blog Post: \n" + blog_post)